In [17]:
#-*-coding:utf-8 -*-

In [8]:
import torch
import tensorflow as tf

from transformers import BertTokenizer
from transformers import BertForSequenceClassification, AdamW, BertConfig
from transformers import get_linear_schedule_with_warmup
#에러 발생시 위 코드 주석으로 대체
#from transformers import WarmupLinearSchedule as get_linear_schedule_with_warmup
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from keras_preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

import pandas as pd
import numpy as np
import random
import time
import datetime

import csv

In [9]:
n_devices = torch.cuda.device_count()
print(n_devices)

0


In [10]:
for i in range(n_devices):
    print(torch.cuda.get_device_name(i))

In [11]:
import csv
import pandas as pd

#데이터 읽어오기, csv파일 위치 추가
chatbot_data = pd.read_csv("CSV 파일 위치")
#데이터 밸런싱을 신경써서 섞기
chatbot_data_shuffled = chatbot_data.sample(frace=1).reset_index(drop=True)
#훈련데이터와 테스트데이터 나누기
train = chatbot_data_shuffled[:9000]
test = chatbot_data_shuffled[9000:]
#문장의 앞마다 [CLS], 문장 종료는 [SEP]를 붙여 인식시킨다.
sentences = ["[CLS] " + str(s) + " [SEP]" for s in train.Q]

#제대로 들어갔나 확인
#sentences[:5]
#0과 1이 들어간 라벨 컬럼은 array에 따로 저장
labels = train['label'].values

TypeError: read_csv() missing 1 required positional argument: 'filepath_or_buffer'

In [ ]:
import pandas as pd
from transformers import BertTokenizer

#BERT에서 사용하는 토크나이저 WordPiece
# 단어를 토큰화할 때, 단어 집합에 없는 단어는 더 쪼개서 ##을 붙여주는 방식
tokenizer = BertTokenizer.from_pretrained("bert-base-multilingual-cased",
                                          do_lower_case=False)

#안녕하세요에 이를 적용
#현재 단어집합에 단어가 없으므로 ##을 붙임으로써 
#녕하세요가 어떠한 단어의 서브워드라는 것을 나타내줌
result = tokenizer.tokenize("안녕하세요")
print(result)

#WordPiece 토크나이저를 이용해 전체 데이터에 토크나이징 수행
##################################################
#어떤 모델이든 단어 자체를 텍스트 형식으로 입력할 수 없으니,
#단어를 토큰으로 만든뒤 정수 인코딩, 패딩 과정 수행
#기본적으로 자주 등장하는 단어를 단어 집합에 추가
##################################################
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased',
                                         do_lower_case=False)
tokenized_texts = [tokenizer.tokenize(s) for s in sentences]

print(sentences[0])  #토크나이징 전
print(tokenized_texts[0]) #토크나이징 후

In [ ]:
# 문장의 최대 시퀀스를 설정하여 정수 인코딩과 제로 패딩을 수행
MAX_LEN = 128 # 최대 시퀀스 길이 설정

input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]
input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long",
                          truncating="post",
                         padding="post")

In [ ]:
attention_masks = []

# 어텐션 마스크
##################################################
## 어텐션이란? 
#쿼리가 주어졌을 때, 이 쿼리와 
#여러 개의 키와의 유사도를 각각 구하고,
#구한 유사도를 가중치로 설정하여 각각의 값을 구한 뒤
#이 값들을 모두 가중합하여 반환하는 함수
######
#단어 벡터에 대해 쿼리, 키 값의 가중치 행렬을 곱해주어
#쿼리, 키, 값 벡터를 얻어낸다.
#그 후 쿼리 벡터에 대해 어텐션 스코어를 구하고,
#이를 이용하여 모든 값 벡터를 가중합하여 어텐션 값을 구하게 된다.
##################################################
## 0 값을 가지는 패딩 토큰에 대해서 
## 어텐션 연산을 불필요하게 수행하지 않도록 단어와 
## 패딩 토큰을 구분할 수 있게 알려주는 것
# 패딩된 데이터가 있을 때 패딩된 값은 0, 패딩되지 않은 단어는 1
#어텐션 마스크 생성
for seq in input_ids:
    seq_mask = [float(i>0) for i in seq]
    atteuntion_masks.append(seq_mask)

In [1]:
import asyncio
import websockets



async def echo(websocket, path):
    async for message in websocket:
        print(message)
        # message를 분석하여 주피터 노트북으로 값을 전달하는 코드 작성
        #분석
        #await websocket.send_text(f"Message text was: {data}")

async def main():
    async with websockets.serve(echo, "localhost", 8765):
        await asyncio.Future()  # 서버가 계속 실행되도록 대기

# 현재 실행 중인 이벤트 루프 가져오기
loop = asyncio.get_event_loop()

# 이벤트 루프에서 비동기 코드 실행
loop.create_task(main())

<Task pending name='Task-4' coro=<main() running at C:\Users\shinheeeul\AppData\Local\Temp\ipykernel_22368\4001136174.py:13>>

test


connection handler failed
Traceback (most recent call last):
  File "C:\Users\shinheeeul\anaconda3\lib\site-packages\websockets\legacy\protocol.py", line 968, in transfer_data
    message = await self.read_message()
  File "C:\Users\shinheeeul\anaconda3\lib\site-packages\websockets\legacy\protocol.py", line 1038, in read_message
    frame = await self.read_data_frame(max_size=self.max_size)
  File "C:\Users\shinheeeul\anaconda3\lib\site-packages\websockets\legacy\protocol.py", line 1113, in read_data_frame
    frame = await self.read_frame(max_size)
  File "C:\Users\shinheeeul\anaconda3\lib\site-packages\websockets\legacy\protocol.py", line 1170, in read_frame
    frame = await Frame.read(
  File "C:\Users\shinheeeul\anaconda3\lib\site-packages\websockets\legacy\framing.py", line 69, in read
    data = await reader(2)
  File "C:\Users\shinheeeul\anaconda3\lib\asyncio\streams.py", line 723, in readexactly
    await self._wait_for_data('readexactly')
  File "C:\Users\shinheeeul\anaconda3